In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine

# Initial Processing

In [2]:
geo = gpd.read_file('data/source/all.json')

In [3]:
a= gpd.read_file('data/source/attributesWithLabels.json')

In [4]:
a

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,pitch,zoom,canvas_x,canvas_y,canvas_width,canvas_height,gsv_url,image_date,label_date,label_severity,label_is_temporary,agree_count,disagree_count,notsure_count,label_tags,label_description,user_id,geometry
0,67742272,NoCurbRamp,5197,6430815,North Beacon Hill,5,False,95,prMAsZ283Q-GBpPv9OA2IQ,143.0625,-21.0625,1,256,180,720,480,https://maps.googleapis.com/maps/api/streetvie...,2018-08,2019-03-12 21:51:50.167,5.0,False,1,1,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30701 47.57400)
1,67742328,NoCurbRamp,2710,6383313,North Beacon Hill,5,False,96,prMAsZ283Q-GBpPv9OA2IQ,143.0625,-21.0625,1,335,171,720,480,https://maps.googleapis.com/maps/api/streetvie...,2018-08,2019-03-12 21:51:55.954,5.0,False,6,1,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30703 47.57397)
2,67742064,NoCurbRamp,5198,6430815,North Beacon Hill,5,False,97,prMAsZ283Q-GBpPv9OA2IQ,212.2500,-26.6875,1,276,163,720,480,https://maps.googleapis.com/maps/api/streetvie...,2018-08,2019-03-12 21:51:59.950,5.0,False,5,2,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30719 47.57398)
3,67742327,NoCurbRamp,5198,6430815,North Beacon Hill,5,False,98,prMAsZ283Q-GBpPv9OA2IQ,212.2500,-26.6875,1,425,161,720,480,https://maps.googleapis.com/maps/api/streetvie...,2018-08,2019-03-12 21:52:01.322,5.0,False,4,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30725 47.57400)
4,67742329,NoCurbRamp,5198,6430815,North Beacon Hill,5,False,99,prMAsZ283Q-GBpPv9OA2IQ,300.9375,-29.8750,1,320,154,720,480,https://maps.googleapis.com/maps/api/streetvie...,2018-08,2019-03-12 21:52:03.355,5.0,False,2,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.30727 47.57410)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21610,67785294,Obstacle,2149,6371594,Georgetown,5,False,233178,MkuZD8ciTTAKUipda9Uy5Q,155.3125,-30.5000,1,329,116,720,480,https://maps.googleapis.com/maps/api/streetvie...,2017-08,2023-02-06 02:02:35.067,5.0,False,0,0,0,"[ ""fire hydrant"" ]",None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.31886 47.54385)
21611,67785502,NoCurbRamp,2149,6371594,Georgetown,5,False,233179,MkuZD8ciTTAKUipda9Uy5Q,155.3125,-30.5000,1,306,125,720,480,https://maps.googleapis.com/maps/api/streetvie...,2017-08,2023-02-06 02:02:42.100,5.0,False,0,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.31886 47.54387)
21612,67785122,SurfaceProblem,2489,6379028,Georgetown,5,False,233186,p7RxYBrekeee00hkwg1lNw,266.0625,-14.9375,1,175,207,720,480,https://maps.googleapis.com/maps/api/streetvie...,2019-07,2023-02-06 02:11:54.317,5.0,False,0,0,0,"[ ""sand\/gravel"" ]",None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32468 47.53719)
21613,67784135,NoSidewalk,6514,6452375,Georgetown,5,False,233189,p8GuSQbQbvyB3yLURw1qoA,359.0625,-20.8125,1,164,166,720,480,https://maps.googleapis.com/maps/api/streetvie...,2021-08,2023-02-06 02:15:46.630,5.0,False,0,0,0,[ ],None,dc9c7b1a-e2bd-4e7f-9d7e-dd239d7c7062,POINT (-122.32512 47.54541)


In [5]:
#drop gsv_panomara_id, correct, high_quality_user column
geo = geo.drop(columns=['gsv_panorama_id','correct','high_quality_user','audit_task_id'])

In [6]:
geo

,label_id,label_type,severity,geometry
0,85055,CurbRamp,1.0,POINT (-122.33279 47.61860)
1,85057,Obstacle,2.0,POINT (-122.33189 47.61781)
2,85059,CurbRamp,1.0,POINT (-122.33183 47.61751)
3,85060,CurbRamp,1.0,POINT (-122.33188 47.61752)
4,85062,CurbRamp,1.0,POINT (-122.33205 47.61764)
...,...,...,...,...
195539,231270,CurbRamp,1.0,POINT (-122.26726 47.54077)
195540,231285,NoSidewalk,5.0,POINT (-122.26344 47.55229)
195541,231338,CurbRamp,1.0,POINT (-122.28179 47.56327)
195542,231339,CurbRamp,1.0,POINT (-122.28165 47.56347)


In [7]:
meta = pd.read_json('data/source/cvMetadata.json')

In [8]:
meta

,label_id,gsv_panorama_id,label_type_id,agree_count,disagree_count,notsure_count,image_width,image_height,sv_image_x,sv_image_y,canvas_width,canvas_height,canvas_x,canvas_y,zoom,heading,pitch,photographer_heading,photographer_pitch
0,85053,3sAn6u8bQPVW3hTDfNUP1w,1,0,0,0,16384.0,8192.0,2484,-390,720,480,62,232,1,107.750000,-15.625000,270.675171,0.168884
1,85004,CpU83mS7vz17EnE02JGOkg,1,0,0,0,16384.0,8192.0,394,-389,720,480,149,254,1,41.750000,-10.375000,129.873657,0.026001
2,85024,RDV4HddwNqwfBZndZRqqkA,1,0,0,0,16384.0,8192.0,3867,-135,720,480,386,187,1,100.437500,-12.625000,128.847916,0.042198
3,85025,NpkOF1LfE024Ks0XIhCttw,1,0,1,0,13312.0,6656.0,5484,-295,720,480,435,222,1,136.062500,-11.500000,350.971405,1.353600
4,85034,-l9SjmKbZmUfr3JV3PuTVg,1,1,0,0,13312.0,6656.0,7887,-504,720,480,584,247,1,180.312500,-15.562500,171.933517,-0.698708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195539,96388,Ek8nn67kKNgCFjZkKf9ISA,7,0,0,0,16384.0,8192.0,12898,-166,720,480,364,259,2,348.645081,-3.973214,318.706665,0.226311
195540,122316,6U9DDI4UppbHDTyp8_J3JA,2,2,0,0,16384.0,8192.0,13246,-636,720,480,327,145,1,3.171875,-32.000000,186.167953,0.891960
195541,122337,lydaO-3xUm3YPy2sNPyPoQ,4,2,0,0,16384.0,8192.0,10316,-614,720,480,302,220,1,288.500000,-20.250000,181.292786,5.152237
195542,122341,ubr500pdt0vImqg71_dHsQ,1,1,0,0,16384.0,8192.0,7437,-322,720,480,377,148,1,198.312500,-23.562500,180.803787,3.150978


In [9]:
meta = meta.drop(columns=['image_width','image_height','sv_image_x','sv_image_y','canvas_width','canvas_height','canvas_x','canvas_y','label_type_id','agree_count','disagree_count','notsure_count'])

In [10]:
#merge metadata and geo data on label_id
df = pd.merge(geo, meta, on='label_id')

In [11]:
#read in label-and-user-ids.csv
user_ids = pd.read_csv('data/source/label-and-user-ids.csv')
# select only city == seattle
user_ids = user_ids[user_ids['city'] == 'seattle']

In [12]:
#megre user_ids and df on label_id
df = pd.merge(df, user_ids, on='label_id', how='inner')

In [13]:
#drop city column
df = df.drop(columns=['city'])

In [14]:
# break geometry into lat and lng
df['lat'] = df['geometry'].apply(lambda x: x.y)
df['lng'] = df['geometry'].apply(lambda x: x.x)


In [15]:
#select lng < -120
df = df[df['lng'] < -120]

In [16]:
#move geomery column to the end
df = df[[c for c in df if c not in ['geometry']] + ['geometry']]
df

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,geometry
0,85055,CurbRamp,1.0,NjPAkwTxWsayAq3kCugvdA,1,107.750000,-15.62500,270.556610,0.907036,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.618599,-122.332787,POINT (-122.33279 47.61860)
1,85057,Obstacle,2.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,38.375000,-21.87500,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617805,-122.331886,POINT (-122.33189 47.61781)
2,85059,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617512,-122.331833,POINT (-122.33183 47.61751)
3,85060,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,131.187500,-24.50000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617519,-122.331879,POINT (-122.33188 47.61752)
4,85062,CurbRamp,1.0,f_Mibt-6nJU7HTN7kx5ZGQ,1,257.187500,-35.00000,164.133438,0.335564,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.617638,-122.332047,POINT (-122.33205 47.61764)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195539,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.00000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26726 47.54077)
195540,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.65625,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344 47.55229)
195541,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.00000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179 47.56327)
195542,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.00000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165 47.56347)


In [19]:
# save to csv
df.to_csv('data/processed-labels/seattle_labels_all.csv', index=False)

In [21]:
#save df to shapefile
df.to_file('data/processed-labels/seattle_labels_all.shp')